# Strategies

This notebook aims to test a neural network using different word embedding methods.

**Word embedding methods:**

| Method       | Definition                                                                 | Advantages                                                                                     | Limitations                                                                 |
|--------------|-----------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| Word2Vec     | Represents words as dense vectors in a continuous space, capturing semantic relationships based on context. | Effective for capturing semantic relationships; widely used in NLP tasks.                     | Requires large datasets; struggles with out-of-vocabulary words.            |
| Glove        | Generates word embeddings by factorizing a co-occurrence matrix, capturing both local and global semantic relationships. | Captures both local and global context; effective for text classification and sentiment analysis. | Computationally expensive; requires pre-computed co-occurrence statistics.  |
| USE (Universal Sentence Encoder) | Produces embeddings for entire sentences rather than individual words, leveraging deep learning models. | Captures sentence-level semantics; pre-trained models available for quick use.                | Higher computational cost; less effective for word-level tasks.             |
| BERT |  |                 |              |

# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import time
import warnings
from collections import Counter
import pickle
from tqdm import tqdm

# NLTK
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

# scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             fbeta_score, make_scorer, matthews_corrcoef, balanced_accuracy_score,
                             classification_report, confusion_matrix, roc_auc_score, roc_curve)


import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Add Hugging Face transformers for BERT
# try:
#     from transformers import BertTokenizer, TFBertModel
# except ImportError:
#     print("Installing transformers library...")
#     import sys
#     !{sys.executable} -m pip install transformers
#     from transformers import BertTokenizer, TFBertModel

print("Version:", tf.__version__)
print("GPU disponible :", tf.config.list_physical_devices('GPU'))


import mlflow
import mlflow.keras


plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'DejaVu Sans'
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ikusawalaetitia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ikusawalaetitia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ikusawalaetitia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Version: 2.16.2
GPU disponible : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)

# Data preparation

## Import

In [4]:
path_to_sample = "../data/processed_sample_tweets.csv"

sample_df = pd.read_csv(path_to_sample, encoding='utf-8')
# Display the first few rows of the dataframe
sample_df.head(10)

,target,ids,date,user,text,processed_text_lem,advanced_processed_text_lem,processed_text_stem,advanced_processed_text_stem
0,0,2065378704,2009-06-07 08:28:25,juliaindelicate,@christt I really wanted to go i didn't get t...,< mention > really wanted go not get go anal b...,< mention > really wanted go not get go anal b...,< mention > realli want go not get go ani anal...,< mention > realli want go not get go ani anal...
1,0,2198776518,2009-06-16 16:33:19,hxcfairy,Finally home from my sisters and I'm so tired,finally home sister tired,finally home sister tired,final home sister tire,final home sister tire
2,0,2258118867,2009-06-20 15:24:59,TranceGemini613,"I really, really wish I'd known all the stuff ...",really really wish would known stuff james tel...,really really wish would known stuff james tel...,realli realli wish would known stuff jame tell...,realli realli wish would known stuff jame tell...
3,0,1755733793,2009-05-10 09:53:26,jwatkins5377,Can officially say I have been at work EVERYDA...,officially say work everyday week ... bad toda...,officially say work everyday week ... bad toda...,offici say work everyday thi week ... bad toda...,offici say work everyday thi week ... bad toda...
4,0,1881082418,2009-05-22 03:32:26,davelee,#FirstRecord The Mr Blobby single Sorry folks.,< hashtag > mr blobby single sorry folk .,< hashtag > mr lobby single sorry folk .,< hashtag > mr blobbi singl sorri folk .,< hashtag > mr lobbi singl sorri folk .
5,0,2045678642,2009-06-05 11:04:38,johnny101,My turkey and ham salad wrap is not sitting we...,turkey ham salad wrap not sitting well .,turkey ham salad wrap not sitting well .,turkey ham salad wrap not sit well .,turkey ham salad wrap not sit well .
6,0,2323712552,2009-06-25 01:17:50,Matt_Whiting,Footy Trainin Was called off Now i Dont kn...,footy trainin wa called dont know ? laugh loud,footy training wa called not know ? lot,footi trainin wa call dont know ? laugh loud,footi train wa call not know ? lot
7,0,2200119346,2009-06-16 18:27:54,JoePaley17,Just got bacl from ftbl summer school tomorrow,got bacl ftbl summer school tomorrow,got back feel summer school tomorrow,got bacl ftbl summer school tomorrow,got back feel summer school tomorrow
8,0,2247872282,2009-06-19 19:45:15,trexsandwich,"@writesfortea Cool teachers are the best, but ...",< mention > cool teacher best never lasted sch...,< mention > cool teacher best never lasted sch...,< mention > cool teacher best never last schoo...,< mention > cool teacher best never last schoo...
9,0,1835988160,2009-05-18 07:09:13,renzzee,"@rediska08 awts. friday, parteeey! hahaha! sa...",< mention > awts . friday parteey ! hahaha ! s...,< mention > awts . friday parteey ! hahaha ! s...,< mention > awt . friday parteey ! hahaha ! sa...,< mention > awt . friday parteey ! hahaha ! sa...


## Split

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X_lem = sample_df['processed_text_lem']
X_stem = sample_df['processed_text_stem']

y = sample_df['target']

# Split the data into training, testing and validation sets
X_temp_lem, X_test_lem, y_temp_lem, y_test_lem = train_test_split(
    X_lem, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
X_train_lem, X_val_lem, y_train_lem, y_val_lem = train_test_split(
    X_temp_lem, y_temp_lem,
    test_size=0.25,  # 0.25 x 0.8 = 0.2
    random_state=42,
    stratify=y_temp_lem
)

# Display the shapes of the resulting datasets
print(f"X_train_lem shape: {X_train_lem.shape}")
print(f"X_test_lem shape: {X_test_lem.shape}")
print(f"y_train_lem shape: {y_train_lem.shape}")
print(f"y_test_lem shape: {y_test_lem.shape}")


# Split the data into training and testing sets
X_temp_stem, X_test_stem, y_temp_stem, y_test_stem = train_test_split(
    X_stem, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
X_train_stem, X_val_stem, y_train_stem, y_val_stem = train_test_split(
    X_temp_stem, y_temp_stem,
    test_size=0.25,  # 0.25 x 0.8 = 0.2
    random_state=42,
    stratify=y_temp_stem
)


# Display the shapes of the resulting datasets
print(f"X_train_stem shape: {X_train_stem.shape}")
print(f"X_val_stem shape: {X_val_stem.shape}")
print(f"X_test_stem shape: {X_test_stem.shape}")
print(f"y_train_stem shape: {y_train_stem.shape}")
print(f"X_val_stem shape: {X_val_stem.shape}")
print(f"y_test_stem shape: {y_test_stem.shape}")

X_train_shape = X_train_lem.shape
X_val_shape = X_val_lem.shape
X_test_shape = X_test_lem.shape

X_train_lem shape: (30000,)
X_test_lem shape: (10000,)
y_train_lem shape: (30000,)
y_test_lem shape: (10000,)
X_train_stem shape: (30000,)
X_val_stem shape: (10000,)
X_test_stem shape: (10000,)
y_train_stem shape: (30000,)
X_val_stem shape: (10000,)
y_test_stem shape: (10000,)


## MLFlow setup

In [7]:
import os
from dotenv import load_dotenv
import time

import mlflow
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient


from tqdm import tqdm


# For scikit-learn
mlflow.sklearn.autolog()

# Configuring MLflow
load_dotenv()
tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(tracking_uri)
print(f"MLflow Tracking URI: {tracking_uri}")

MLflow Tracking URI: http://localhost:8080


In [20]:
# Create a new MLflow Experiment
experiment_name = "P7-Sentiments_Analysis_neural_network"

# Check if the experiment exists, create it if it doesn't
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Created new experiment '{experiment_name}' with ID: {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Using existing experiment '{experiment_name}' with ID: {experiment_id}")

# Set the experiment
mlflow.set_experiment(experiment_name)

Created new experiment 'P7-Sentiments_Analysis_neural_network' with ID: 915290206608231433


<Experiment: artifact_location='mlflow-artifacts:/915290206608231433', creation_time=1758900314087, experiment_id='915290206608231433', last_update_time=1758900314087, lifecycle_stage='active', name='P7-Sentiments_Analysis_neural_network', tags={}>

# Functions

In [29]:
class TweetVectorizer:
    def __init__(self, preprocessor: str = None, vectoriser: str = 'w2v'):
        self.preprocessor = preprocessor
        self.vectoriser = vectoriser
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()

        # Initialize vectorizers
        if vectoriser == 'w2v':
            from gensim.models import Word2Vec
            self.model = None # Will be initialized during fit
        elif vectoriser == 'fasttext':
            from gensim.models import FastText
            self.model = FastText(vector_size=100, window=5, min_count=1, workers=4)
        elif vectoriser == 'use':
            import tensorflow_hub as hub
            self.model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        elif vectoriser == 'bert':
            # Initialize BERT tokenizer and model
            from transformers import BertTokenizer, TFBertModel
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            self.model = TFBertModel.from_pretrained('bert-base-uncased')
        else:
            raise ValueError("Unsupported vectoriser. Choose from 'w2v', 'fasttext', 'use', or 'bert'.")

    def preprocess(self, text: str) -> str:
        """
        Cleans and preprocesses a single text string by replacing URLs, mentions, and hashtags,
        converting to lowercase, removing special characters, and removing stopwords.

        Args:
            text (str): The text string to process.

        Returns:
            str: The processed text string.
        """


        if pd.isna(text):
            return ""
        # Replace URLs with <URL>
        processed = re.sub(r'https?://\S+', '<URL>', text)
        # Replace mentions with <MENTION>
        processed = re.sub(r'@[A-Za-z0-9_]+', '<MENTION>', processed)
        # Separate # from word and replace the word with <HASHTAG>
        processed = re.sub(r'#([A-Za-z0-9_]+)', r'#<HASHTAG>', processed)

        # Convert text to lowercase
        processed = processed.lower()

        # Remove special characters and numbers, keeping !, ?, and ellipsis (...)
        # Also keeps the placeholders <URL>, <MENTION>, <HASHTAG>
        processed = re.sub(r'[^a-z0-9\s.!?<>#]', '', processed)

        # Tokenize the text
        tokens = word_tokenize(processed)

        # Initialize lemmatizer
        if self.preprocessor == 'lemmatization':
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        # Initialize stemmer
        if self.preprocessor == 'stemming':
            tokens = [self.stemmer.stem(token) for token in tokens]


        # Define negative words that should not be removed
        negative_words = {
            'no', 'not', 'nor', "don't", "aren't", "couldn't", "didn't", "doesn't",
            "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
            "needn't", "shan't", "shouldn't", "wasn't", "weren't", "won't", "wouldn't",
            "never", "none", "nobody", "nothing", "nowhere", "neither"
        }
        # Create a set of stopwords to remove, excluding the negative words
        stop_words_to_remove = set(stopwords.words('english')) - negative_words

        # remove stopwords, and join back to string
        filtered_tokens = [word for word in tokens if word not in stop_words_to_remove]

        return ' '.join(filtered_tokens)

    def fit_transform(self, texts: pd.Series):
        """
        Fits the vectorizer model (if applicable) and transforms the input texts into vectors.

        Args:
            texts (pd.Series): Series of text strings to fit and transform.

        Returns:
            np.ndarray: Array of vectorized texts.
        """
        processed_texts = texts.apply(self.preprocess).tolist()

        if self.vectoriser == 'w2v':
            from gensim.models import Word2Vec
            tokenized_texts = [text.split() for text in processed_texts]
            self.model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)
            vectors = np.array([np.mean([self.model.wv[word] for word in text.split() if word in self.model.wv] or [np.zeros(100)], axis=0) for text in processed_texts])
        elif self.vectoriser == 'fasttext':
            tokenized_texts = [text.split() for text in processed_texts]
            self.model.build_vocab(tokenized_texts, update=True)
            self.model.train(tokenized_texts, total_examples=len(tokenized_texts), epochs=self.model.epochs)
            vectors = np.array([np.mean([self.model.wv[word] for word in text.split() if word in self.model.wv] or [np.zeros(100)], axis=0) for text in processed_texts])
        elif self.vectoriser == 'use':
            vectors = self.model(processed_texts).numpy()
        elif self.vectoriser == 'bert':
            # For BERT, we need to tokenize the text and return the encoded features
            # This returns a batch of token IDs, attention masks, and token type IDs
            encoded_inputs = self.tokenizer(
                processed_texts,
                padding='max_length',
                truncation=True,
                max_length=self.max_length,
                return_tensors='tf'
            )

            # Get BERT embeddings - using CLS token (first token) as the sentence representation
            outputs = self.model(encoded_inputs)
            # We'll use the CLS token (first token) embeddings as the sentence representation
            vectors = outputs.last_hidden_state[:, 0, :].numpy()  # Shape: (batch_size, hidden_size=768)
        else:
            raise ValueError("Unsupported vectoriser. Choose from 'w2v', 'fasttext', 'use', or 'bert'.")

        # Chech for 2D array
        if vectors.ndim == 1:
            vectors = vectors.reshape(-1, 1)
        elif vectors.ndim == 2 and vectors.shape[1] == 1:
            vectors = vectors.reshape(-1, vectors.shape[1])

        return vectors

    def transform(self, texts: pd.Series):
        """
        Transforms the input texts into vectors using the fitted vectorizer model.

        Args:
            texts (pd.Series): Series of text strings to transform.

        Returns:
            np.ndarray: Array of vectorized texts.
        """
        processed_texts = texts.apply(self.preprocess).tolist()

        if self.vectoriser == 'w2v':
            if self.model is None:
                raise ValueError("The Word2Vec model has not been fitted. Call fit_transform first.")
            vectors = np.array([np.mean([self.model.wv[word] for word in text.split() if word in self.model.wv] or [np.zeros(100)], axis=0) for text in processed_texts])
        elif self.vectoriser == 'fasttext':
            if self.model is None:
                raise ValueError("The FastText model has not been fitted. Call fit_transform first.")
            vectors = np.array([np.mean([self.model.wv[word] for word in text.split() if word in self.model.wv] or [np.zeros(100)], axis=0) for text in processed_texts])
        elif self.vectoriser == 'use':
            vectors = self.model(processed_texts).numpy()
        elif self.vectoriser == 'bert':
            # For BERT, we need to tokenize the text and return the encoded features
            encoded_inputs = self.tokenizer(
                processed_texts,
                padding='max_length',
                truncation=True,
                max_length=self.max_length,
                return_tensors='tf'
            )

            # Get BERT embeddings - using CLS token (first token) as the sentence representation
            outputs = self.model(encoded_inputs)
            vectors = outputs.last_hidden_state[:, 0, :].numpy()  # Shape: (batch_size, hidden_size=768)
        else:
            raise ValueError("Unsupported vectoriser. Choose from 'w2v', 'fasttext', 'use', or 'bert'.")

        return vectors

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional  # Ajout de Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec, FastText

from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

class LSTMTweetClassifier:
    def __init__(self, embedding:str, embedding_dim: int, lstm_units: int=128, max_length: int = 100):
        self.embedding = embedding
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.max_length = max_length
        self.tokenizer = Tokenizer()
        self.model = None
        self.word_index = self.tokenizer.word_index

    def build_embedding_matrix(self):
        if self.embedding == 'w2v':
            from gensim.models import Word2Vec
            embedding_model = Word2Vec.load("word2vec.model")
        elif self.embedding == 'fasttext':
            from gensim.models import FastText
            embedding_model = FastText.load("fasttext.model")
        else:
            raise ValueError("Unsupported embedding type. Choose 'w2v' or 'fasttext'.")

        vocab_size = len(self.word_index) + 1  # +1 for padding token
        embedding_matrix = np.zeros((vocab_size, self.embedding_dim))

        for word, i in self.word_index.items():
            if word in embedding_model.wv:
                embedding_matrix[i] = embedding_model.wv[word]
            else:
                embedding_matrix[i] = np.random.normal(size=(self.embedding_dim,))

        return embedding_matrix

    def build_model(self, vocab_size: int, embedding_matrix: np.ndarray):
        """Builds the LSTM model architecture.

        Args:
            vocab_size (int): Size of the vocabulary.
            embedding_matrix (np.ndarray): Pre-trained embedding matrix.
        Returns:
            keras.Model: Compiled LSTM model.
        """
        # Initialize the model
        model = Sequential()
        # Add embedding layer with pre-trained weights
        model.add(Embedding(input_dim=vocab_size,
                            output_dim=self.embedding_dim,
                            weights=[embedding_matrix],
                            input_length=self.max_length,
                            trainable=False))  # Freezing the embedding layer
        # Add a Bidirectional LSTM layer
        model.add(Bidirectional(LSTM(self.lstm_units, return_sequences=False)))
        # Add a Dense output layer with sigmoid activation for binary classification
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                      optimizer=Adam(learning_rate=0.001),
                      metrics=['accuracy'])

        self.model = model
        return model

    def get_metrics(self, y_true, y_pred):
        """Calculates and returns various classification metrics.

        Args:
            y_true (np.ndarray): True labels.
            y_pred (np.ndarray): Predicted labels.
        Returns:
            dict: Dictionary containing accuracy, precision, recall, f1-score, and MCC.
        """
        accuracy = np.mean((y_pred > 0.5).flatten == y_true)
        specificity = np.sum((y_pred < 0.5).flatten & (y_true == 0)) / np.sum(y_true == 0)
        auc = roc_auc_score(y_true, y_pred)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("specificity", specificity)
        mlflow.log_metric("auc", auc)
        return {
            'accuracy': accuracy,
            'specificity': specificity,
            'auc': auc
        }

    def train_and_evaluate(self, X_train, y_train, X_val, y_val, epochs=10, batch_size=32):
        """Trains the LSTM model and evaluates it on the validation set.

        Args:
            X_train (np.ndarray): Training feature vectors.
            y_train (np.ndarray): Training labels.
            X_val (np.ndarray): Validation feature vectors.
            y_val (np.ndarray): Validation labels.
            epochs (int, optional): Number of training epochs. Defaults to 10.
            batch_size (int, optional): Batch size for training. Defaults to 32.
        Returns:
            dict: Dictionary containing evaluation metrics on the validation set.
        """
        if self.model is None:
            raise ValueError("The model has not been built. Call build_model first.")
        with mlflow.start_run(run_name=f"LSTM_{self.embedding}_{self.epochs}epochs_{self.lstm_units}units", nested= True):
            # log model parameters
            mlflow.log_param("embedding", self.embedding)
            mlflow.log_param("embedding_dim", self.embedding_dim)
            mlflow.log_param("lstm_units", self.lstm_units)
            mlflow.log_param("max_length", self.max_length)
            mlflow.log_param("epochs", epochs)
            mlflow.log_param("batch_size", batch_size)

            start_time = time.time()
            history = self.model.fit(X_train, y_train,
                                     validation_data=(X_val, y_val),
                                     epochs=epochs,
                                     batch_size=batch_size,
                                     verbose=1)
            training_time = time.time() - start_time
            mlflow.log_metric("training_time", training_time)

            # Log training and validation metrics for each epoch
            for epoch in range(epochs):
                mlflow.log_metric("train_loss", history.history['loss'][epoch], step=epoch)
                mlflow.log_metric("train_accuracy", history.history['accuracy'][epoch], step=epoch)
                mlflow.log_metric("val_loss", history.history['val_loss'][epoch], step=epoch)
                mlflow.log_metric("val_accuracy", history.history['val_accuracy'][epoch], step=epoch)

            # Evaluate on validation set
            start_prediction_time = time.time()
            y_val_pred = self.model.predict(X_val).flatten()
            prediction_time = time.time() - start_prediction_time
            mlflow.log_metric("prediction_time", prediction_time)

            # Calulate validation metrics
            val_metrics = {
                'accuracy': accuracy_score(y_val, (y_val_pred > 0.5).astype(int)),
                'precision': precision_score(y_val, (y_val_pred > 0.5).astype(int)),
                'recall': recall_score(y_val, (y_val_pred > 0.5).astype(int)),
                'f1_score': f1_score(y_val, (y_val_pred > 0.5).astype(int)),
                'specificity': np.sum((y_val_pred < 0.5).flatten() & (y_val == 0)) / np.sum(y_val == 0),
                'roc_auc': roc_auc_score(y_val, y_val_pred),
                'prediction_time': prediction_time
            }

            mlflow.log_table(pd.DataFrame([val_metrics]), "validation_metrics.csv")
            for metric_name, metric_value in val_metrics.items():
                mlflow.log_metric(metric_name, metric_value)

            # ROC AUC proba
            y_val_proba = self.model.predict(X_val).flatten()
            fpr, tpr, thresholds = roc_curve(y_val, y_val_proba)
            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % val_metrics['roc_auc'])
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver Operating Characteristic')
            plt.legend(loc="lower right")
            plt.savefig("roc_curve.png")
            mlflow.log_artifact("roc_curve.png")
            plt.close()

            # Log the model
            mlflow.keras.log_model(self.model, f"LSTM_{self.embedding}_{self.lstm_units}units_model")
            run_id = mlflow.active_run().info.run_id
            result= mlflow.register_model(f"runs:/{run_id}/LSTM_{self.embedding}_{self.lstm_units}units_model", f"LSTM_{self.embedding}_{self.lstm_units}units_model")

    def fit(self, text: pd.Series, labels: pd.Series, test_size:float = 0.2, val_split:float = 0.2,epochs: int=10, batch_size: int=32):
        # Tokenize and pad sequences
        self.tokenizer.fit_on_texts(text)
        sequences = self.tokenizer.texts_to_sequences(text)
        padded_sequences = pad_sequences(sequences, maxlen=self.max_length, padding='post', truncating='post')

        vocab_size = len(self.tokenizer.word_index) + 1  # +1 for padding token

        # Build embedding matrix
        embedding_matrix = self.build_embedding_matrix()

        # Build the model
        self.build_model(vocab_size, embedding_matrix)

        # split the data into training/val and test sets
        X_temp, X_test, y_temp, y_test = train_test_split(
            padded_sequences, labels, test_size=test_size, random_state=42, stratify=labels
        )

        # Split into train set into train and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_split, random_state=42, stratify=y_temp)

        print(f"X_train shape: {X_train.shape}")
        print(f"X_val shape: {X_val.shape}")
        print(f"X_test shape: {X_test.shape}")

        # Train and evaluate the model
        self.epochs = epochs
        self.train_and_evaluate(padded_sequences, labels, X_val, y_val, epochs, batch_size)

        # Final evaluation on the test set
        y_test_pred = self.model.predict(X_test).flatten()
        test_metrics = self.get_metrics(y_test, y_test_pred)
        print("Test Metrics:", test_metrics)
        for metric_name, metric_value in test_metrics.items():
            mlflow.log_metric(f"test_{metric_name}", metric_value)

    def predict(self, texts: pd.Series):
        """Predicts the sentiment of the input texts.

        Args:
            texts (pd.Series): Series of text strings to predict.

        Returns:
            np.ndarray: Array of predicted labels.
        """
        if self.model is None:
            raise ValueError("The model has not been built or trained. Call fit first.")

        # Preprocess and tokenize the input texts
        sequences = self.tokenizer.texts_to_sequences(texts)
        padded_sequences = pad_sequences(sequences, maxlen=self.max_length, padding='post', truncating='post')

        # Predict using the trained model
        predictions = self.model.predict(padded_sequences).flatten()
        predicted_labels = (predictions > 0.5).astype(int)

        return predicted_labels


# Training

In [17]:
config =[
    {
        'embedding': 'w2v',
        'embedding_dim': 100,
        'lstm_units': 128,
        'max_length': 100,
        'epochs': 5,
        'batch_size': 32
    },
    {
        'embedding': 'fasttext',
        'embedding_dim': 100,
        'lstm_units': 128,
        'max_length': 100,
        'epochs': 5,
        'batch_size': 32
    }
]

In [ ]:
def train_and_save_gensim_models(tokenized_corpus: pd.Series):
    # --- Train and Save Word2Vec Model ---
    print("Training Word2Vec model...")
    w2v_model = Word2Vec(sentences=tokenized_corpus,
                        vector_size=100,  # Should match 'embedding_dim' in your config
                        window=5,
                        min_count=1,
                        workers=4)
    w2v_model.save("word2vec.model")
    print("Word2Vec model saved as word2vec.model")


    # --- Train and Save FastText Model ---
    print("\nTraining FastText model...")
    ft_model = FastText(sentences=tokenized_corpus,
                        vector_size=100,  # Should match 'embedding_dim' in your config
                        window=5,
                        min_count=1,
                        workers=4)
    ft_model.save("fasttext.model")
    print("FastText model saved as fasttext.model")

In [ ]:
experiment_name = "P7-Sentiments_Analysis_LSTM"
document = sample_df["text"]
labels = sample_df["target"]
validation_metrics = []

for cfg in config:
    with mlflow.start_run(run_name=f"LSTM_{cfg['embedding']}_{cfg['epochs']}epochs_{cfg['lstm_units']}units_model", nested=False):

        # Initialize the vectorizer
        vectorizer = TweetVectorizer(preprocessor=cfg['embedding'], vectoriser=cfg['embedding'])

        print(f"Training LSTM with {cfg['embedding']} embeddings...")
        model = LSTMTweetClassifier(embedding=cfg['embedding'],
                                    embedding_dim=cfg['embedding_dim'],
                                    lstm_units=cfg['lstm_units'],
                                    max_length=cfg['max_length'])

        # Training
        model.fit(document, labels, epochs=cfg['epochs'], batch_size=cfg['batch_size'])

        mlflow.end_run()
        print(f"Completed training for {cfg['embedding']} embeddings.")

Training LSTM with w2v embeddings...
🏃 View run LSTM_w2v_5epochs_128units_model at: http://localhost:8080/#/experiments/915290206608231433/runs/e614b4dcd5f04fb78bb3c95e0071fd8f
🧪 View experiment at: http://localhost:8080/#/experiments/915290206608231433


FileNotFoundError: [Errno 2] No such file or directory: 'word2vec.model'